# Learning Objectives

In this lab, we will learn about:

* Understanding intra-node GPU topology and interconnections like PCIe and NVLink
* Architecture overview of NVIDIA DGX system with 8 Tesla V100s and DGX system with 8 Ampere A100s
* Comparison of communication links and their impact on application performance
* p2pBandwidthLatencyTest micro-benchmark for P2P performance analysis.

# Intra-Node Communication Topology

Let's dive deeper into how the underlying communication architecture of our system affects program performance. Run the command below to display your node's GPU and NIC communication topology(due to limited resources we try on 2 GPUs only)::

In [ ]:
!srun --partition=gpu -n1 --gres=gpu:2 nvidia-smi topo -m

If the output is unclear, you can launch a Terminal session by clicking on `File` $\rightarrow$ `New` $\rightarrow$ `Terminal` or by following the steps as shown:

<!---![open_terminal_session](../../images/open_terminal_session.png)-->

![open_terminal_session](../../images/open_terminal_session_jupyter_lab.png)

### DGX system with 8 Tesla V100

Output of running the command:
    
<img src="../../images/nvidia_smi_topo_output.png">
    
Focus on a particular row, say GPU 0. The output states that GPUs 1 through 4 are connected to it via NVLink (in addition to PCIe) and GPUs 5 through 7 are connected to it via PCIe as well as an "SMP" interconnect. We have a dual-socket system and the CPUs in these sockets are connected by an interconnect known as SMP interconnect.

Thus, GPU 0 to GPU 5 communication happens via not just PCIe, but also over the inter-socket interconnect within the same node. Clearly, this is a longer path than say the one between GPU 0 and GPU 1, which are connected via NVLink directly. We will discuss the NIC to GPU connection in the inter-node section of this bootcamp.

Even within the GPUs connected via NVLink, we see different annotations such as `NV1` and `NV2` that affect the communication bandwidth and hence the performance. In this section, we will explore the nuances associated with a diverse intra-node GPU communication topology like in the output above. Specifically, the communication topology is as follows:

<img src="../../images/dgx1_8x_tesla_v100_topo.png" width="70%" height="70%">

### DGX system with 8 Ampere A100
Output of running the command:
    
<img src="../../images/nvidia_smi_topo_output_A100.png">

In constrast to the DGX system with 8 V100s, GPU0 to GPU7 are connected via NVLink(NV12) and in summary, each GPU in A100 is connected to all other GPUs via the third generation NVLink. As a result, this doubles the GPU-to-GPU direct bandwidth to 600 gigabytes per second (GB/s), almost 10X higher than PCIe Gen 4 with a new NVIDIA NVSwitch that’s 2X faster than the V100. The communication topology is as follows:

<img src="../../images/dgx_a100_topo.jpg" width="70%" height="70%">

Qualitatively, the bandwidth and latency vary with the topology as follows:

<img src="../../images/intra_node_topology_map.png" width="50%" height="50%">

Host staging implies traversing through the CPU and the travel path taken is one of PHB, NODE, and SYS. In contrast, if the path taken is either NV1, NV2, or PIX, then P2P is available. PXB implies that the GPUs belong to different PCIe hubs and P2P is usually not supported in this case.

A double NVLink connection provides twice the bandwidth compared to a single NVLink. 

For a pair of 2 GPUs, the peak bidirectional bandwidth are as follows:
* PCIe: Using PIX topology, 15.75GB/s for PCIe Gen 3.0 and 31.5GB/s for PCIe Gen 4.0.
* NVLink: Using NV# topology, 50GB/s per connection. So a double NVLink connection has 100GB/s peak bidirectional bandwidth.

Let us understand what difference the underlying communication topology can make to the application performance in the following sub-section.

**Note:** If your command output doesn't show any NVLink connection or if there's no difference in connection type (PIX, PXB, PHB, NODE, SYS, NV#) between any 2 pair of GPUs, then the communication bandwidth and latency will likely be the same between any pair and the following sub-sections will not display any performance difference.

## Performance variation due to system topology

So far, the code runs the multi-GPU version on all available GPUs in a node (8 in our case). We can supply the `-gpus` runtime flag to the binary to run our code on specific GPUs. If we want to run on only 2 GPUs, namely GPU 0 and GPU 3, we use the `-gpus 0,3` argument. 

In [ ]:
!cd ../../source_code/cuda && srun --partition=gpu -n1 --gres=gpu:4 ./jacobi_memcpy -p2p -gpus 0,3

### DGX system with 8 Ampere A100
On a DGX system with 8 A100s, choosing any GPU pairs would result in the same output as the P2P is available between all GPU 0 to 7.

Output when choosing GPU 0,3:

```bash
Num GPUs: 2. Using GPU ID: 0, 3, 
16384x16384: 1 GPU:   3.9853 s, 2 GPUs:   2.0641 s, speedup:     1.93, efficiency:    96.54 
```

Output when choosing GPU 0,7:
```bash
Num GPUs: 2. Using GPU ID: 0, 7, 
16384x16384: 1 GPU:   3.9835 s, 2 GPUs:   2.0638 s, speedup:     1.93, efficiency:    96.51 
```

Now remove the `-p2p` flag and run the command again for one the pairs (GPUs 0 and 7 in this example) and you will notice a change in performance as the underlying communication path changes.

```bash
Num GPUs: 2. Using GPU ID: 0, 7, 
16384x16384: 1 GPU:   3.9849 s, 2 GPUs:   2.1221 s, speedup:     1.88, efficiency:    93.89 
```

Feel free to reduce the grid size using `-nx` and `-ny` flags (to say 8192$\times$8192) and see the effect on efficiency. If we remove the `-p2p` flag and and run the command again for GPUs 0 and 7, we will not get any difference in performance on DGX A100 system. As you may recall, P2P is not possible between GPUs 0 and 7, so the underlying communication path doesn't change, resulting in same performance with and without the `-p2p` flag. This can be confirmed by profiling the application and looking at the operations performed in the Nsight Systems timeline. 


### DGX system with 8 Tesla V100
On a DGX system with 8 V100s, try to find the GPU pair with highest bandwidth available as per the table above and replace `0,3` with those GPUs, and then re-run the above command. The efficiency would likely be higher than before due to less inter-GPU communication (each GPU does more wok instead). Our output is as follows:

```bash
Num GPUs: 2. Using GPU ID: 0, 3, 
16384x16384: 1 GPU:   4.4513 s, 2 GPUs:   2.2664 s, speedup:     1.96, efficiency:    98.20  
```

Now, run the binary a pair of GPUs that have the lowest available bandwidth. In our case, we use GPU 0 and GPU 7. Our output is:

```bash
Num GPUs: 2. Using GPU ID: 0, 7, 
16384x16384: 1 GPU:   4.4529 s, 2 GPUs:   2.3454 s, speedup:     1.90, efficiency:    94.93  
```

![p2p_2_gpu_memcpy_nsys](../../images/p2p_2_gpu_memcpy_nsys.png)

Feel free to reduce the grid size using `-nx` and `-ny` flags (to say 8192$\times$8192) and see the effect on efficiency. 

## Benchmarking the system topology

Our application is not very memory intensive. As is visible from the profiler output, $\gt95\%$ of the time in GPU is spent on computation. Therefore, to get a quantitative measure of latency and bandwidth impact due to topology, we run a micro-benchmark.

### p2pBandwidthLatencyTest micro-benchmark

p2pBandwidthLatencyTest is a part of [CUDA Samples GitHub repository](https://github.com/NVIDIA/cuda-samples) available to help CUDA developers. 

As the name suggests, this test measures the bandwidth and latency impact of P2P and underlying communication topology. Let's compile the benchmark:

In [ ]:
!cd ../../source_code/p2pBandwidthLatencyTest/ && make clean && make

Now, let's run the benchmark:

In [ ]:
!cd ../../source_code/p2pBandwidthLatencyTest/ && srun --partition=gpu -n1 --gres=gpu:2 ./p2pBandwidthLatencyTest

The first part of the benchmark gives device information and P2P access available from each GPU (similar to `nvidia-smi topo -m` command). Next, the benchmark measures the unidirectional and bidirectional bandwidth and latency with P2P disabled and enabled.

### DGX system with 8 Ampere A100
Partial results obtained from a DGX system with 8 A100s:

```bash
Bidirectional P2P=Disabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 1549.47  18.06  19.47  19.20  19.23  19.14  19.13  18.90 
   
Bidirectional P2P=Enabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 1548.13 424.16 424.70 425.01 424.52 425.18 420.90 425.42 
     
P2P=Disabled Latency Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   2.89  24.68  24.58  24.58  24.60  24.59  24.67  24.59 
     
P2P=Enabled Latency (P2P Writes) Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   2.86   3.47   3.44   3.42   3.36   3.36   3.37   3.43 
```

The above values show GPU-to-GPU unidirectional transfer bandwidth ranging from 270GB/s to 275GB/s. Bidirectional bandwidth ranges from 425GB/s to 519GB/s. Within the same GPU (diagonal output), shows a bandwidth within the same GPU of around 1,580GB/s. These numbers are close to the theoretical full 600GB/s bidirectional NVLink bandwidth that can be achieved between pairs of GPUs.

### DGX system with 8 Tesla V100
Partial results obtained from a DGX system with 8 Tesla V100s:

```bash
Bidirectional P2P=Disabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 783.95   9.56  14.43  14.46  14.47  14.24  14.51  14.43 

Bidirectional P2P=Enabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3      4      5      6      7 
     0 784.87  48.49  48.49  96.85  96.90  14.25  14.54  14.49 
     
P2P=Disabled Latency Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   1.78  17.52  16.41  16.43  17.35  16.88  17.34  16.85 
     
P2P=Enabled Latency (P2P Writes) Matrix (us)
   GPU     0      1      2      3      4      5      6      7 
     0   1.76   1.62   1.61   2.01   2.02  18.44  19.15  19.34
```

Our DGX system with 8 Tesla V100s is based on PCIe gen 3.0 with a peak maximum GPU-GPU PCIe banwidth of 15.75 GB/s. Let us analyze and understand these results:

* GPU 0 and GPU 1/2: Connected by a single NVLink connection. By enabling P2P-
  - Bandwidth reaches close to the maximum peak of 50 GB/s.
  - Latency decreases by an order of magnitude.
* GPU 0 and GPU 3/4: Connected by a double NVLink connection. By enabling P2P-
  - Bandwidth reaches close to the maximum peak of 100 GB/s.
  - Latency decreases by an order of magnitude.
* GPU 0 and GPU 5/6/7: Connected by PCIe and SMP interconnect. By enabling P2P- 
  - Bandwidth is unchanged.
  - Latency increases a marginally.
  

If you correlate these results with the communication topology that can be displayed by usng `nvidia-smi topo -m` command and the qualtitative table in the previous section, they should be consistent with one another. For such systems, we should try to set the GPUs in an application such that a GPU can share data with its neighbours using a high-bandwidth, low-latency communication topology. Enabling P2P, when possible, usually improves the performance by eliminating host staging.


We now have an in-depth understanding of intra-node topology and its effects on performance. Let us now analyze our P2P-enabled application again to uncover opportunities to extract more performance.

Click on the link below to access the next lab where we discuss the need for CUDA streams and then implement them in our application.

# [Next: CUDA Streams](../cuda/streams.ipynb)

Here's a link to the home notebook through which all other notebooks are accessible:

# [HOME](../../../start_here.ipynb)

---
## Links and Resources

* [Documentation: NVIDIA DGX 1 Tesla V100 Whitepaper](https://images.nvidia.com/content/pdf/dgx1-v100-system-architecture-whitepaper.pdf)
* [Concepts: NVLink](https://www.nvidia.com/en-in/data-center/nvlink/)
* [Research: Effect of topology-awareness on communication](https://ieeexplore.ieee.org/abstract/document/7529932)
* [Code: p2pBandwidthLatencyTest](https://github.com/NVIDIA/cuda-samples/tree/master/Samples/p2pBandwidthLatencyTest)
* [Code: Multi-GPU Programming Models](https://github.com/NVIDIA/multi-gpu-programming-models)
* [Code: GPU Bootcamp](https://github.com/gpuhackathons-org/gpubootcamp/)

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

## Licensing
Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.
